In [1]:
import cvxpy as cp
import numpy as np

In [ ]:
n = 3
C = cp.Parameter((n,n), value=np.array([[ 0.78453811, -1.32050747,  0.57919997],
       [ 0.44379913,  0.58134071,  1.03254784],
       [ 0.7311226 ,  0.66465688, -0.2464693 ]]))
c = cp.Parameter((n,1), value=np.array([1,1,1]).reshape((-1,1)), nonneg=True)

A = cp.Parameter((n,n), value=np.array([[-0.99639688, -0.2216588 ,  1.06555159],
       [ 2.00884919,  0.01563207, -0.36271868],
       [-0.9113235 , -0.12149785, -0.24807337]]))
B = cp.Parameter((n,n), value=np.array([[-1.42367144, -0.62673584, -0.45906866],
       [ 0.6591621 ,  0.96948298,  1.1490893 ],
       [ 0.70077944, -1.33992307, -0.17423502]]))
b = cp.Parameter((n,1), value=np.array([[ 1.72769017],
       [-0.97577213],
       [ 1.02798925]]))

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), hermitian=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    A[i][0] * cp.real(X[i][0]) + B[i][0] * cp.imag(X[i][0]) <= b[i] for i in range(n)
]
constraints += [cp.real(X) <= np.ones((n,n))]
# prob = cp.Problem(cp.Minimize(cp.trace(C @ cp.real(X))), constraints)
prob = cp.Problem(cp.Minimize(cp.sum(
    [c[i]*cp.square(cp.real(X[i][1])) + c[i]*cp.real(X[i][2]) for i in range(n)]
)), constraints)

# prob.is_dcp()

prob.solve(solver=cp.CVXOPT, verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

# Rank reduction

In [57]:
n = 4

C_orig = np.eye(n)
C = cp.Parameter((n,n), value=np.copy(C_orig))

X = cp.Variable((n,n), PSD=True)

constraints = []
for i in range(n):
    for j in range(i+1, n):
        e_i = np.eye(1,n,i)
        e_j = np.eye(1,n,j)
        e_ij = (e_i - e_j).reshape(-1,1)
        A = e_ij @ e_ij.reshape(1,-1)
        constraints += [ cp.trace(A @ X) >= 1 ]

prob = cp.Problem(cp.Minimize(cp.trace(C @ X)), constraints)

# prob.is_dcp()

prob.solve(solver=cp.MOSEK, verbose=False)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)
print()

print("rank(X) =", (np.linalg.matrix_rank(X.value, tol=1e-6)))
print()

print("eigenvalues:")
print(np.linalg.eig(X.value)[0])
print("eigenvectors:")
print(np.linalg.eig(X.value)[1])

The optimal value is 1.499999997706409
A solution X is
[[ 0.375 -0.125 -0.125 -0.125]
 [-0.125  0.375 -0.125 -0.125]
 [-0.125 -0.125  0.375 -0.125]
 [-0.125 -0.125 -0.125  0.375]]

rank(A) = 3

eigenvalues:
[ 4.99999999e-01 -3.05537621e-10  4.99999999e-01  4.99999999e-01]
eigenvectors:
[[ 0.8660254  -0.5         0.02551494  0.14574682]
 [-0.28867513 -0.5         0.53738335 -0.22022986]
 [-0.28867513 -0.5        -0.8068732  -0.64374409]
 [-0.28867513 -0.5         0.24397491  0.71822714]]


In [58]:
v = np.linalg.eig(X.value)[1].T[0]  # 0, 3, 1

In [59]:
# v = np.linalg.eig(X.value)[1].T[0]  # 0, 3, 1
C.value += 3.5*v.reshape(-1,1) @ v.reshape(1,-1)
# print("The new cost matrix is")
# print(C.value)

prob.solve(solver=cp.MOSEK, verbose=False)

# Print result.
print("A new solution X is")
print(X.value)
print()
print("The objective value in the original problem is", np.trace(C_orig @ X.value))

print("rank(X) =", (np.linalg.matrix_rank(X.value, tol=1e-6)))
print()

print("eigenvalues:")
print(np.linalg.eig(X.value)[0])
print("eigenvectors:")
print(np.linalg.eig(X.value)[1])

A new solution X is
[[ 1.19056282e-12 -2.68674979e-13 -2.84581912e-13 -2.78050588e-13]
 [-2.68674979e-13  1.00000000e+00 -5.00000000e-01 -5.00000000e-01]
 [-2.84581912e-13 -5.00000000e-01  1.00000000e+00 -5.00000000e-01]
 [-2.78050588e-13 -5.00000000e-01 -5.00000000e-01  1.00000000e+00]]

The objective value in the original problem is 2.99999999999606
rank(A) = 2

eigenvalues:
[1.46766623e-12 3.59258069e-13 1.50000000e+00 1.50000000e+00]
eigenvectors:
[[-8.66024694e-01  5.00001229e-01 -6.07290360e-15  4.45700915e-15]
 [ 2.88675844e-01  4.99999590e-01 -7.97976114e-01  1.71694151e-01]
 [ 2.88675844e-01  4.99999590e-01  2.49236292e-01 -7.77143567e-01]
 [ 2.88675844e-01  4.99999590e-01  5.48739822e-01  6.05449416e-01]]


In [22]:
n = 3
C = cp.Parameter((n,n), value=np.array([[ 0.78453811, -1.32050747,  0.57919997],
       [ 0.44379913,  0.58134071,  1.03254784],
       [ 0.7311226 ,  0.66465688, -0.2464693 ]]))
c = cp.Parameter((n,1), value=np.array([1,1,1]).reshape((-1,1)), nonneg=True)

A = cp.Parameter((n,n), value=np.array([[-0.99639688, -0.2216588 ,  1.06555159],
       [ 2.00884919,  0.01563207, -0.36271868],
       [-0.9113235 , -0.12149785, -0.24807337]]))
B = cp.Parameter((n,n), value=np.array([[-1.42367144, -0.62673584, -0.45906866],
       [ 0.6591621 ,  0.96948298,  1.1490893 ],
       [ 0.70077944, -1.33992307, -0.17423502]]))
b = cp.Parameter((n,1), value=np.array([[ 1.72769017],
       [-0.97577213],
       [ 1.02798925]]))

# Create a symmetric matrix variable.
X = cp.Variable((n,n), hermitian=True)

constraints = [ cp.quad_over_lin(cp.norm(X[i][j]), cp.real(X[i][i])) <= cp.real(X[j][j]) for i,j in [(0,1), (0,2), (1,2)]]
constraints += [
    A[i][0] * cp.real(X[i][0]) + B[i][0] * cp.imag(X[i][0]) <= b[i] for i in range(n)
]
constraints += [cp.real(X) <= np.ones((n,n))]

prob = cp.Problem(cp.Minimize(cp.sum(
    [c[i]*cp.square(cp.real(X[i][1])) + c[i]*cp.real(X[i][2]) for i in range(n)]
)), constraints)

# print(prob.is_dcp())

prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is -0.2507304200506858
A solution X is
[[ 0.99999999+0.00000000e+00j -0.35074946+4.11384939e-01j
  -0.59609004-5.16125480e-06j]
 [-0.35074946-4.11384939e-01j  0.29226277+0.00000000e+00j
  -0.3222542 +0.00000000e+00j]
 [-0.59609004+5.16125480e-06j -0.3222542 +0.00000000e+00j
   0.35532334+0.00000000e+00j]]


In [17]:
X = cp.Variable((2,2), hermitian=True)
# z^2/y <= x
constraints = [cp.quad_over_lin(cp.norm(X[0][1]), cp.real(X[1][1])) <= cp.real(X[0][0]), cp.real(X[1][1]) <= 2, cp.real(X[0][1]) >= 1 ]
prob = cp.Problem(cp.Minimize(cp.real(X[0][0])), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.5000000046832747

In [6]:
x = cp.Variable()
y = cp.Variable()
z = cp.Variable()
# z^2/y <= x
constraints = [cp.quad_over_lin(z, y) <= x, y <= 2, z >= 1 ]

prob = cp.Problem(cp.Minimize(x), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.5000000136111296

In [7]:
xyz = cp.Variable(shape=(3,))  # first two coordinates are "x" and "y"
# xy >= z^2
t = cp.Variable()
aux_constraint = t >= cp.norm(xyz[2])
lmi_constraint = cp.bmat([[xyz[0], t],[t, xyz[1]]]) >> 0
constraints = [aux_constraint, lmi_constraint, xyz[1] <=2, xyz[2] >= 1]

prob = cp.Problem(cp.Minimize(xyz[0]), constraints)

print(prob.is_dcp())
print(prob.is_dpp())

prob.solve()
prob.value

True
True


0.49999999981725113